In [1]:
"""
Data Loading.
"""

'\nData Loading.\n'

In [95]:
import os
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import sys
import warnings
warnings.filterwarnings("ignore")

In [4]:
%%time
stops_path = "./Data/Transit_Stops/Transit_Stops_for_King_County_Metro__transitstop_point.shp"
stops = gpd.read_file(stops_path)

CPU times: user 13.4 s, sys: 271 ms, total: 13.6 s
Wall time: 13.4 s


In [5]:
%%time
signals_path = "./Data/Signals/KCM_signals.shp"
signals = gpd.read_file(signals_path)

CPU times: user 227 ms, sys: 12.7 ms, total: 240 ms
Wall time: 228 ms


In [6]:
%%time
kc_streets_path = "./Data/Seattle_Streets/KCM_Streets.shp"
kc_streets = gpd.read_file(kc_streets_path)

CPU times: user 52.1 s, sys: 922 ms, total: 53.1 s
Wall time: 52.1 s


In [7]:
%%time
kcm_routes_path = "./Data/Routes/Comprehensive_Routes/KCM_Routes.shp"
kcm_routes = gpd.read_file(kcm_routes_path)

CPU times: user 253 ms, sys: 0 ns, total: 253 ms
Wall time: 252 ms


In [8]:
%%time
accel_prof_path = "./Data/Accel_Prof/acceleration.csv"
raw_acc_prof = pd.read_csv(accel_prof_path, names=['time[s]', 'accel.[g]'])

CPU times: user 2.48 ms, sys: 0 ns, total: 2.48 ms
Wall time: 1.94 ms


In [9]:
%%time
all_stops = stops.copy()

all_signals = signals['geometry'].reset_index(drop = True)

all_limits = kc_streets[['geometry','SPEED_LIM']]


CPU times: user 21.2 ms, sys: 0 ns, total: 21.2 ms
Wall time: 19.9 ms


In [92]:
# Display a map of the route overlayed over area data:
# Import area map:
seattle = gpd.read_file("./Data/Census_Tracts_2000/Census_Tracts_2000.shp")

# filter out all 'water' zones in the data for clarity
seattle = seattle[(seattle['WATER'] != 1)]

In [128]:
def separate_period_ridership(rider_df):
    '''
    separate_period_ridership splits a dataframe containing ridership information,
    and then generates the mean ridership change per stop for a given period.
    
    '''
    dict_list = {}
    for period in rider_df['Period'].unique():
        period_ridership = rider_df[rider_df['Period']==period]
        sorted_riders = period_ridership.sort_values(by=['InOut','Trip_ID', 'STOP_SEQ', 'STOP_ID'])
        sorted_riders = sorted_riders[['InOut','STOP_SEQ', 'STOP_ID', 'AveOn', 'AveOff', 'AveLd']]
        sorted_riders = sorted_riders.drop_duplicates(subset=['InOut','STOP_SEQ'], keep='first')
        sorted_riders = sorted_riders.drop_duplicates(subset='STOP_ID', keep='first')
        sorted_riders = sorted_riders.reset_index(drop = True)
        sorted_riders['Index'] = sorted_riders.index

        # get the mean ridership change at each stop
        sorted_riders_diff = sorted_riders['AveOn'] - sorted_riders['AveOff']
        sorted_riders['diff']=sorted_riders_diff
        
        dict_list[period] = sorted_riders
        
    return dict_list

In [141]:
def separate_used_stops(rider_sep, stops):
    dict_list = {}
    for period in rider_sep:
        sorted_riders = rider_sep[period]
        copy = stops.copy()
        copy['ROUTE_LIST'].fillna(value = str(0), inplace = True)
        rider_change_stops = copy[copy['STOP_ID'].isin(list(sorted_riders['STOP_ID']))]
        rider_change_stops = rider_change_stops[rider_change_stops["IN_SERVICE"] == "Y"]
        dict_list[period] = rider_change_stops.reset_index(drop=True)
    return dict_list

In [142]:
import os
import Helper_Methods as hm
#variable for route sampling:
route_concat_val = 6 # every 6th point in the data is used

# get the available route shapefiles
route_data_dir = "./Data/Routes/Individual_Routes/"
routefiles_raw = pd.Series(os.listdir(route_data_dir))
routefiles = routefiles_raw[routefiles_raw.apply(lambda x: ('shp' in (x.split("."))[-1]))].reset_index(drop='true')
routefiles = routefiles.apply(lambda x: "{}{}".format(route_data_dir, x))

# get the ridership data
ridership_path = "./Data/Trip_Data/Zon183Unsum.csv"
kcm_ridership = pd.read_csv(ridership_path)

data_dict_list = {}

print("Available Routes:")
for route in routefiles:
    route_shape = gpd.read_file(route)
    route_sampled = route_shape[::route_concat_val].reset_index(drop=True)
    route_geometry = route_sampled['geometry']
    route_elevation = route_sampled['Z']
    route_num = (route.split('rt')[-1]).split("_")[0]
    rider_df = kcm_ridership[(kcm_ridership['Route']==int(route_num))]
    rider_sep = separate_period_ridership(rider_df)
    stops_sep = separate_used_stops(rider_sep, all_stops)
    #route_stops = hm.get_stops(all_stops, str(route_num))['geometry'].reset_index(drop=True)
    data_dict = {"geometry":route_geometry,
                 "elevation":route_elevation,
                 "ridership":rider_sep,
                 "stops":stops_sep}
    data_dict_list[route_num] = data_dict
    print("\t{}".format(route_num))

Available Routes:
	22
	186
	101


In [120]:
filt_101 = kcm_ridership[(kcm_ridership['Route']==int(186))]
period_ridership = filt_101[filt_101['Period']=='XNT']
sorted_riders = period_ridership.sort_values(by=['InOut','Trip_ID', 'STOP_SEQ', 'STOP_ID'])
sorted_riders = sorted_riders[['InOut','STOP_SEQ', 'STOP_ID', 'AveOn', 'AveOff', 'AveLd']]
sorted_riders = sorted_riders.drop_duplicates(subset=['InOut','STOP_SEQ'], keep='first')
sorted_riders = sorted_riders.drop_duplicates(subset='STOP_ID', keep='first')
sorted_riders = sorted_riders.reset_index(drop = True)
sorted_riders['Index'] = sorted_riders.index

# get the mean ridership change at each stop
sorted_riders_diff = sorted_riders['AveOn'] - sorted_riders['AveOff']
rider_stops = sorted_riders['STOP_ID'].unique()
matched_stops = []
for stop in stop_stops:
    if stop in rider_stops:
        matched_stops.append(stop)
print(len(matched_stops), len(rider_stops), len(stop_stops))

19 19 70


In [108]:
copy = all_stops.copy()
copy['ROUTE_LIST'].fillna(value = str(0), inplace = True)
rider_change_stops = copy[copy['STOP_ID'].isin(list(sorted_riders['STOP_ID']))]
rider_change_stops = rider_change_stops[rider_change_stops['ROUTE_LIST'].str.split(" ").apply(lambda x: '186' in x)]['STOP_ID']

In [110]:
len(rider_change_stops)

38

In [20]:
import RouteMap as rm
saved_list = os.listdir('./Data/SAVED_ROUTEMAPS/')
for route in data_dict_list.keys():
    route_data = data_dict_list[route]
    for period in route_data['stops'].keys():
        routemap_name = 'route_map_{}_{}.csv'.format(route, period)
        route_map = None
        if routemap_name in saved_list:
            print('saved route found: {}'.format(routemap_name))
            route_map = rm.RouteMap(route_data['geometry'], route_data['elevation'])
            route_map = route_map.load_from_gdf('./Data/SAVED_ROUTEMAPS/{}'.format(routemap_name))
        else:
            print('generating route for {}'.format(route))
            route_map = rm.RouteMap(route_data['geometry'],
                                    route_data['elevation'],
                                    all_limits,
                                    route_data['stops'],
                                    all_signals,
                                    debug = True)
            route_map.get_gdf().to_csv('./Data/SAVED_ROUTEMAPS/{}'.format(routemap_name))
    route_data['map']=route_map

saved route found: route_map_22.csv
saved route found: route_map_186.csv
saved route found: route_map_101.csv


In [25]:
import BusModel as bm
acc_prof_path = "./Data/Accel_Prof/acceleration.csv"
raw_acc_prof = pd.read_csv(acc_prof_path, names=['time[s]', 'accel.[g]'])

# Make a duplicate to alter
a_prof = raw_acc_prof.copy()
# Convert g's to m/s^2 and then drop g's
a_prof['accel.[m/s^2]'] = a_prof['accel.[g]'] * 9.81
a_prof = a_prof.drop(columns = ['accel.[g]'])

default_bus = bm.BusModel(a_prof)

In [26]:
import TripDynamics as td

### TripDynamics Debugging

In [40]:
import random
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import RouteMap as rm

class TripDynamics:
    def __init__(self,
                 route_map,
                 bus_model,
                 bus_ridership,
                 braking_aggresion = 1, # default 1 based on maximum braking aggression
                 ridership_type='mean', # means how the ridership is taken, mean passengers on/off, or chance of passenger on/off. Chance currently unsure how to use.
                 seed=42, # life, the universe, and everything
                 stoplight_chance = .84615): # Defuault based on 120s cycle, 55s g, 65 red/yellow. https://wsdot.wa.gov/travel/operations-services/traffic-signals:
        self._route_map = route_map
        self._bus_model = bus_model
        self._ridership_type = ridership_type 
        self._rand_seed = seed
        self._braking_aggression = braking_aggresion 
        self._stoplight_chance = stoplight_chance
        
        # Check ridership type
        if (self._ridership_type == 'mean'):
            self._ridership = bus_ridership
        elif (self._ridership_type == 'chance'):
            self._ridership = self.riders_from_chance(bus_ridership)
        
        # generate gdf, ridership, and bus mass
        self._route_data = self._route_map.get_gdf()
        self._route_data['riders'] = self.generate_rider_profile()
        self._route_data['bus_mass'] = self.generate_mass_profile()
        
        self._trip_data = None
        
    
    def generate_rider_profile(self):
        '''
        generate_rider_profile generates a pandas series that contains the
        current amount of riders based on bus position
        
        Parameters:
        None
        
        Returns:
        pandas series matching length of route, with ridership numbers.
        Indicies identical to the route.
        '''
        
        # Copy the ridership changes as d_riders
        d_riders = self._ridership.copy()
        
        # get the series of stop booleans for the route
        stop_bools = self._route_data['is_stop'].copy()
        
        # create a new dataframe based on the stop booleans
        df = pd.DataFrame(stop_bools, columns=['is_stop'])
        
        # default change in riders is 0
        df['d_riders'] = 0.0
        
        # get a list of indexes on the route that are stops
        stop_indexes = list(df[df['is_stop']==True].index)
        
        # At the stops, add the change in riders
        df.iloc[stop_indexes, df.columns.get_loc('d_riders')] = np.asarray(d_riders, dtype=np.float32)
        
        # make the cumulative riders and return it
        df['riders'] = df['d_riders'].cumsum()
        
        # Assign a class variable for ridership change
        self._ridership_change = df['d_riders']
        
        return df['riders']
    
    

    def generate_mass_profile(self):
        '''
        generate_mass_profile generates a pandas series
        containing the total bus mass at each point along
        the trip, based on the ridership information of the route.
        
        Parameters:
        None
        
        Returns:
        a pandas series containing the total bus mass
        at each point on the route, indexed the same.
        '''
        
        # Copy the stops and riders
        stop_bools = self._route_data['is_stop'].copy()
        n_riders = self._route_data['riders'].copy()
        
        # Generate a new dataframe from the stop booleans
        df = pd.DataFrame(stop_bools, columns=['is_stop'])
        
        # Add the default bus mass to the dataframe
        df['total_mass'] = self._bus_model.get_mass()
        df['total_mass'] = df['total_mass'].astype('float')
        
        
        # add the ridership column to the dataframe
        df['n_riders'] = n_riders
        
        # get stop indexes
        stop_indexes = df[df['is_stop']==True].index
        
        # Get the column indexes of mass and riders
        mass_col_id = df.columns.get_loc('total_mass')
        rider_col_id = df.columns.get_loc('n_riders')
        
        # Loop through each stop,
        for i in range(len(stop_indexes)):
            
            # Get the current route point index
            cur_idx = stop_indexes[i]
            
            # get the ridership value for this index
            rider_val = df.iloc[cur_idx, rider_col_id]
            
            # Set the bus model's riders to the rider value
            self._bus_model.update_riders(rider_val, 'set')
            
            # Get the mass of the bus model
            df.iloc[cur_idx:, mass_col_id] = self._bus_model.get_mass()
        
        # return the total mass series
        return df['total_mass']

    
    def get_mass_profile(self):
        return self._route_data['bus_mass']
    

    def bus_trip(self):
        '''
        bus_trip uses the route data and bus information to simulate the bus
        travelling according to a set logic, and returns the route geodataframe with new information
        on acceleration, time, and velocity.
        
        Parameters:
        None
        
        Returns:
        the geodataframe of all data involved in the process
        '''
        
        # get the route geodataframe
        route = self._route_map.get_gdf()
        
        # get the frictional acceleration profile of the route
        fric_a_prof = self._route_map.get_fric_accel()
        
        # get the hill accelerational profile of the route
        hill_a_prof = self._route_map.get_hill_accel()
        
        # get the bus model
        bus = self._bus_model
        
        # get the friction coefficient of the bus model
        bus_f_coef = bus.get_fric_coeff()
        
        # get the maximum velocity for the defined bus
        max_v = bus.max_velocity()
        
        # variable for how hard the driver presses on the brake while braking
        braking_factor = self._braking_aggression
        
        # define new columns for velocity, stop distance, state, and stopping distance,
        # Power needed, and time change.
        route['vel.[m/s]'] = 0
        route['acc.[m/s^2]'] = 0
        route['stop_dist[m]'] = 0
        route['stppn_dist'] = 0
        route['power_needed[W]'] = 0
        route['st'] = 0
        route['time_change[s]'] = 0
        route['r_change'] = abs(self._ridership_change) > 0
        
        # Generate empty lists to hold the same values as above.
        vel_list = []
        acc_list = []
        dist_list = []
        stp_d_ls = []
        power = []
        st_ls = []
        dt_ls = []
        
        # get the accelerational profile of the bus.
        accel_profile = bus.get_accel_profile()
        
        # Set the random seed
        random.seed(self._rand_seed)
        
        # Set up the final point of the route so that the bus actually stops there.
        route.at[len(route)-2,'is_stop'] = True
        route.at[len(route)-2, 'r_change'] = True
        
        # Loop through each point on the route
        for i in range(1, len(route)-1):
            
            # update the bus's mass.
            current_mass = self._route_data['bus_mass'][i]
            bus.update_mass(current_mass)
            
            # get the current point distance in meters
            point_dist = route['point_distances[km]'][i]*1000 # convert to meters
            
            # get the cumulative distance travelled to this point in meters
            cum_dist = route['cumulative_distance[km]'][i]*1000 # convert to meters
            
            # get the cumulative distance column id
            cum_dist_col_id = route.columns.get_loc('cumulative_distance[km]')
            
            # get the dataframe containing the remaining points that have not been visited
            remaining_trip = route.iloc[i:]
            
            # set the distance to the stop to a dummy variable of 0
            dist_to_stop = 0
            
            # set the index of the next stop to dummy variable of 0
            next_stop_index = 0
            
            # Set up a boolean for checking if the bus will stop. range based on closest integer to the inverse fraction provided
            is_stopping = (random.randrange(int(round(1/self._stoplight_chance),0)) == 0)
            
            # get the list of stops as defined by the distance between each stop on the route,
            #including signals, in meters
            #                                    Check if it's a stop and that stop has a ridership change         or      check if its a signal and the bus is stopping
            stops_remain = remaining_trip[((remaining_trip['is_stop'] == True) & remaining_trip['r_change'] == True)| ((remaining_trip['is_signal'] == True) & is_stopping)]['cumulative_distance[km]'].reset_index(drop=True)*1000 #convert to meters

            # if there are remaining stops,
            if (len(stops_remain) != 0):
                
                # set the distance to the stop to be the difference between the stop distance
                # and cumulative distance
                dist_to_stop = stops_remain[0]-cum_dist # meters
                
                # set the next stop index, handle errors where for some reason there is no value for cumulative distance.
                next_stop_match = route['cumulative_distance[km]'][route['cumulative_distance[km]'] == stops_remain[0]/1000]
                if (len(next_stop_match) != 0):
                    next_stop_index = route['cumulative_distance[km]'][route['cumulative_distance[km]'] == stops_remain[0]/1000].index[0]
                else:
                    # if there's nothing, assume the next stop index is in the next 10.
                    next_stop_index = i+10
                
            # get the current velocity of the bus
            start_velocity = bus.velocity()  
            
            # get the acceleration of bus friction at the point
            a_fric = fric_a_prof[i]*bus_f_coef
            
            # get the acceleration due to gravity at the point
            a_hill = hill_a_prof[i]
            
            # combine the accelerations to get external (to the bus's motor) acceleration
            ext_a = a_fric + a_hill 
            
            # calculate the stopping distance based on the starting velocity and external acceleration
            stopping_dist = bus.get_braking_distance(start_velocity, braking_factor, ext_a) #meters
            
            # get the speed limit at current point
            point_sp_lim = route['speed_limit[km/s]'][i] * 1000 # meters
            
            # Error correction for when speed limit has noise
            stable_limit = True
            if (i > 2 and i < len(route)-2):
                # get a boolean for if any change in speed limit is stable:
                mean_limit = (route['speed_limit[km/s]'][i-1:i+1]*1000).mean()
                stable_limit = abs(point_sp_lim - mean_limit) < .1
                point_sp_lim = mean_limit
            
            # set up a variable for status
            status = ""
            d_power = 0
            d_t = 0
            
            # Driving logic: -------------------------------------------------------
            #If at rest, accelerate for the distance between this and the next point
            if (start_velocity < 0.1):
                status = "accel_from_0"
                d_power, d_t = bus.accelerate_v5(point_dist, ext_a)
                
            # If the distance difference between stopping distance and distance to the stop
            # is less than half the point distance resolution, then brake
            
            elif ((dist_to_stop < (stopping_dist + point_dist))): #and (abs(self._ridership_change[next_stop_index]) > 0)):
                status = "Stopping_brake"
                d_power, d_t = bus.brake_v3(point_dist, braking_factor, ext_a)
            #elif((dist_to_stop-stopping_dist)<= point_dist):
                
            # If the starting velocity is less than the speed limit, accelerate. Margin is 1/25th of speed limit.
            elif(start_velocity < (point_sp_lim - point_sp_lim/25)):
                status = "speed_lim_accel"
                d_power, d_t = bus.accelerate_v5(point_dist, ext_a)
                
            # if the starting celocity is greater than the speed limit, 
            elif(start_velocity > (point_sp_lim+point_sp_lim/25)):
                status = "speed_lim_brk"
                
                # brake for that distance
                d_power, d_t = bus.brake_v3(point_dist, braking_factor-.4, ext_a)
            else:
                status = "maintain_v"
                d_power, d_t = bus.maintain_v5(point_dist, ext_a)
            # End driving logic ---------------------------------------------------------
            
            
            # Append all the statuses and data to their lists
            st_ls.append(status)
            vel_list.append(bus.velocity())
            acc_list.append(bus.get_acceleration())
            dist_list.append(dist_to_stop)
            stp_d_ls.append(stopping_dist)
            power.append(d_power)
            dt_ls.append(d_t)
        
        # convert the lists and apply them to the respective columns
        route.iloc[1:-1, route.columns.get_loc('vel.[m/s]')] = vel_list
        route.iloc[1:-1, route.columns.get_loc('acc.[m/s^2]')] = acc_list
        route.iloc[1:-1, route.columns.get_loc('stop_dist[m]')] = dist_list
        route.iloc[1:-1, route.columns.get_loc('stppn_dist')] = stp_d_ls
        route.iloc[1:-1, route.columns.get_loc('power_needed[W]')] = power
        route.iloc[1:-1, route.columns.get_loc('st')] = st_ls
        route.iloc[1:-1, route.columns.get_loc('time_change[s]')] = dt_ls
        
        route['elapsed_time[s]'] = route['time_change[s]'].cumsum()
        
        # Set the route GDF to an instance variable, then return it.
        self._trip_data = route
        return route
    
    
    def get_all_data(self):
        '''
        get_all_data returns the dataframe generated after a trip is performed.
        
        Parameters:
        N/A
        
        Returns: 
        Dataframe of Trip data, or None if no trip has been performed.
        '''
        if (self._trip_data is None):
            print("Warning, bus_trip() method has not yet been used!")
        return self._trip_data
      
        
    def generate_ESS_profile(self):
        '''
        generate_ESS_profile() is a method to create the Energy Storage System
        power profile based on the required power. Note: This requires bus_trip()
        to be run prior.
        
        Parameters:
        N/A
        
        Returns:
        Pandas series containing required power from the ESS.
        '''
        # Get the trip data's power requirements
        power_profile = self.get_all_data()['power_needed[W]']
        
        # Apply the instance power method to the power profile.
        ESS_profile = power_profile.apply(self.calc_instance_power)
        
        # Return the profile.
        return ESS_profile
    
    
    def calc_instance_power(self, value):
        '''
        calc_instance_power takes in a power value,
        and converts it to the corresponding load on the ESS.
        
        Parameters:
        value: a power value in Watts, as an int or float.
        
        Returns:
        converted battery power as a float.
        '''
        # Get the bus model parameters.
        motor_eff = self._bus_model.get_motor_eff()
        invert_eff = self._bus_model.get_invert_eff()
        aux_eff = self._bus_model.get_auxill_efficiency()
        aux_load = self._bus_model.get_aux_load()
        regen_eff = self._bus_model.get_regen_eff()
        max_regen = -100000
        
        # set the battery power to zero.
        bat_pow = 0
        
        # Including Auxilliary load, though not strictly important at the moment. 
        if (value >= 0):
            # Discharging, converting the needed power into power battery must exert
            bat_pow = value/(motor_eff*invert_eff) + (aux_load/aux_eff)
        elif(value*regen_eff*motor_eff > max_regen):
            #charging, the regenerative braking ALL the time, max regen is 100
            bat_pow = value*regen_eff*motor_eff + (aux_load/aux_eff)
        else:
            bat_pow = max_regen + (aux_load/aux_eff)
            
            
        # Return the battery power.
        return bat_pow
    
    
    def export_trip_data(self):
        '''
        export_trip_data generates a geodataframe with the most relevant informaton for
        ease of use and understanding. For a hollistic report, get_all_data is better suited.
        
        Parameters:
        N/A
        
        Returns:
        geodataframe containing trip geometry, elevation, distance, grade, ridership, mass, velocity,
        elapsed time, and power needed in Watts. 
        '''
        # Drop non-relevant columns.
        raw_trip_data = self.get_all_data().copy()
        raw_trip_data.drop("stop_dist[m]", axis='columns', inplace=True)
        raw_trip_data.drop("stppn_dist", axis='columns', inplace=True)
        raw_trip_data.drop("power_needed[W]", axis='columns', inplace=True)
        raw_trip_data.drop("is_stop", axis='columns', inplace=True)
        raw_trip_data.drop("is_signal", axis='columns', inplace=True)
        raw_trip_data.drop("point_distances[km]", axis='columns', inplace=True)
        raw_trip_data.drop("latitude", axis='columns', inplace=True)
        raw_trip_data.drop("longitude", axis='columns', inplace=True)
        raw_trip_data.drop("speed_limit[km/s]", axis='columns', inplace=True)
        
        # Add the ESS profile.
        raw_trip_data['power_needed[W]'] = self.generate_ESS_profile()
        
        # return the trip data.
        return raw_trip_data

### Run TripDynamics

In [27]:
data_dict_list['22'].keys()

dict_keys(['geometry', 'elevation', 'ridership', 'stops', 'map'])

In [47]:
for route in list(data_dict_list.keys())[:]:
    smellev = data_dict_list[route]['map'].smooth_elevation()
    prev_elev = data_dict_list[route]['map'].get_elevation()
    smoothed_elev_map = data_dict_list[route]['map'].set_elevation(smellev)
    data_dict_list[route]['trip']={}
    for period in list(data_dict_list[route]['ridership'].keys())[:]:
        try:
            print(route, period)
            riders = data_dict_list[route]['ridership'][period]
            print(len(riders))
            print(len(data_dict_list[route]['map'].get_stops()))
            print()
            bus = bm.BusModel(a_prof)
            trip = TripDynamics(smoothed_elev_map, bus, riders)
            data_dict_list[route]['trip'][period]=trip
        except:
            print("Error with {}, {}".format(route, period))
            print(riders)

    reg_elev_map = data_dict_list[route]['map'].set_elevation(prev_elev)
    

22 XEV
86
86

22 AM
86
86

22 MID
86
86

22 PM
86
86

186 AM
70
70

186 XEV
70
70

186 PM
70
70

186 MID
2
70

Error with 186, MID
0   -0.4
1   -8.7
dtype: float64
186 XNT
19
70

Error with 186, XNT
0     1.0
1     0.0
2     0.0
3     0.9
4     0.0
5     1.2
6     0.0
7     0.2
8     0.0
9     0.2
10    0.0
11    0.4
12    0.0
13   -0.1
14    0.0
15    0.0
16    0.0
17    0.0
18    0.1
dtype: float64
101 MID
42
38

Error with 101, MID
0      7.0
1      0.2
2      0.4
3      0.1
4      0.2
5      6.7
6      0.7
7      5.0
8      0.3
9      0.4
10     1.2
11     1.0
12    -0.9
13    -0.6
14     1.0
15     0.1
16    -2.4
17    -2.5
18    -4.9
19   -10.5
20    -2.6
21     5.2
22    16.8
23     8.7
24     4.9
25    -0.8
26    -0.5
27    -0.5
28     0.1
29     1.7
30     0.5
31    -0.8
32    -0.2
33    -0.8
34    -9.6
35    -3.4
36    -0.2
37   -12.9
38    -0.1
39    -0.1
40    -0.3
41    -7.8
dtype: float64
101 PM
42
38

Error with 101, PM
0      4.4
1      0.2
2      0.3
3      0.0
4      

In [51]:
data_dict_list['22']['ridership']

{'XEV': 0     0.0
 1    -0.1
 2    -0.4
 3     0.1
 4     0.0
      ... 
 81    0.0
 82    0.1
 83    0.5
 84    0.2
 85    0.0
 Length: 86, dtype: float64,
 'AM': 0     0.0
 1     0.0
 2     0.0
 3     0.2
 4     0.0
      ... 
 81    0.0
 82   -0.4
 83   -0.2
 84    1.0
 85    0.0
 Length: 86, dtype: float64,
 'MID': 0     0.1
 1     0.2
 2    -0.1
 3     0.7
 4     0.0
      ... 
 81    0.0
 82   -0.2
 83   -0.2
 84    0.1
 85    0.0
 Length: 86, dtype: float64,
 'PM': 0    -0.6
 1    -2.5
 2     0.3
 3    -0.5
 4    -0.3
      ... 
 81    0.0
 82   -0.2
 83    1.6
 84   -0.8
 85   -0.1
 Length: 86, dtype: float64}